In [1]:
import pandas as pd

In [2]:
campaign = pd.read_csv("CAMPAIGN.csv")

In [6]:
campaign.columns

Index(['フィールド名/Field Name', '日本語表示名称/Japanese Display Name',
       '英語表示名称/English Display Name', 'XML属性/XML Attribute', '型/Type',
       '表示回数0回のレコード取得指定可否/Supports Zero Impressions',
       '組合せ不可フィールド/Not compatible fields', '動作区分/Behavior Type'],
      dtype='object')

# 組み合わせ不可フィールドのグループごとに, フィールドをリスト化する

In [292]:
field_list = campaign.groupby("組合せ不可フィールド/Not compatible fields")["フィールド名/Field Name"].apply(list)
field_df = pd.DataFrame(field_list).reset_index()
field_df


,組合せ不可フィールド/Not compatible fields,フィールド名/Field Name
0,"[ALL_CONV, ALL_CONV_RATE, ALL_CONV_VALUE, COST...",[HOUR_OF_DAY]
1,"[ALL_CONV_RATE, AVG_CPC, AVG_DELIVER_RANK, BUD...","[OBJECT_OF_CONVERSION_TRACKING, CONVERSION_NAME]"
2,"[BUDGET_LOST_IMPRESSION_SHARE, EXACT_MATCH_IMP...",[CLICK_TYPE]
3,"[CLICK_TYPE, CONVERSION_NAME, OBJECT_OF_CONVER...","[IMPRESSION_SHARE, EXACT_MATCH_IMPRESSION_SHAR..."
4,"[CONVERSION_NAME, HOUR_OF_DAY, OBJECT_OF_CONVE...","[INVALID_CLICKS, INVALID_CLICK_RATE, ALL_CONV_..."
5,"[CONVERSION_NAME, OBJECT_OF_CONVERSION_TRACKING]","[COST, IMPS, CLICKS, CLICK_RATE, AVG_CPC, AVG_..."
6,[HOUR_OF_DAY],"[ALL_CONV, ALL_CONV_VALUE, VALUE_PER_ALL_CONV,..."


# 組み合わせ不可な要素があるフィールドの数

In [287]:
len(campaign[campaign["組合せ不可フィールド/Not compatible fields"].notna()])

24

In [134]:
import itertools
import numpy as np
import ast

In [128]:
#　組み合わせ不可フィールドの列の要素が文字列として認識されいて使いづらいのでリストに変換しておく
to_list = lambda x: x.replace("[", "").replace("]", "").split(", ")
field_df.loc[:,"ncf"] = field_df["組合せ不可フィールド/Not compatible fields"].apply(to_list)

In [130]:
field_df

,組合せ不可フィールド/Not compatible fields,フィールド名/Field Name,ncf
0,"[ALL_CONV, ALL_CONV_RATE, ALL_CONV_VALUE, COST...",[HOUR_OF_DAY],"[ALL_CONV, ALL_CONV_RATE, ALL_CONV_VALUE, COST..."
1,"[ALL_CONV_RATE, AVG_CPC, AVG_DELIVER_RANK, BUD...","[OBJECT_OF_CONVERSION_TRACKING, CONVERSION_NAME]","[ALL_CONV_RATE, AVG_CPC, AVG_DELIVER_RANK, BUD..."
2,"[BUDGET_LOST_IMPRESSION_SHARE, EXACT_MATCH_IMP...",[CLICK_TYPE],"[BUDGET_LOST_IMPRESSION_SHARE, EXACT_MATCH_IMP..."
3,"[CLICK_TYPE, CONVERSION_NAME, OBJECT_OF_CONVER...","[IMPRESSION_SHARE, EXACT_MATCH_IMPRESSION_SHAR...","[CLICK_TYPE, CONVERSION_NAME, OBJECT_OF_CONVER..."
4,"[CONVERSION_NAME, HOUR_OF_DAY, OBJECT_OF_CONVE...","[INVALID_CLICKS, INVALID_CLICK_RATE, ALL_CONV_...","[CONVERSION_NAME, HOUR_OF_DAY, OBJECT_OF_CONVE..."
5,"[CONVERSION_NAME, OBJECT_OF_CONVERSION_TRACKING]","[COST, IMPS, CLICKS, CLICK_RATE, AVG_CPC, AVG_...","[CONVERSION_NAME, OBJECT_OF_CONVERSION_TRACKING]"
6,[HOUR_OF_DAY],"[ALL_CONV, ALL_CONV_VALUE, VALUE_PER_ALL_CONV,...",[HOUR_OF_DAY]


In [288]:
# 全通りを試すためのシミュレーション
def simulation():
    good_pattern = {}
    num_val = 7
    r = range(1,7)
    # 可能なリクエストの組み合わせを作成する(組み合わせ時に組み合わせ不可のフィールド同士が組み合わないようにする)
    for i in r:
        good_pattern[str(i)] = []
        print("+--------------------------------------------+")
        for (a, b) in zip(itertools.combinations(field_df["フィールド名/Field Name"], i), itertools.combinations(field_df["ncf"], i)):
            if(set(np.concatenate(a)).isdisjoint(set(np.concatenate(b)))):
                good_pattern[str(i)].append(np.concatenate(a))
    #可能なリクエストのパターンを手とした手数i
    for i in r:
         # 当該手数のときのリクエストパターンの組み合わせ
        for l in itertools.combinations(r, i):
             # 当該手数の合計数
            if sum(l) == 7: 
                # リクエストパターン同士で直積を作り、かぶりがなければ24フィールドすべてを満たしている
                for products in itertools.product(*[good_pattern[str(a)] for a in l]): 
                    if intersection(products) == set() :
                        print(l)
                        print(products)
                    
                        


In [282]:
def intersection(lists):
    sets = list(map(set, lists))
    result = sets.pop()
    for s in sets:
        result = result.intersection(s)
    return result

In [289]:
simulation()

+--------------------------------------------+
+--------------------------------------------+
+--------------------------------------------+
+--------------------------------------------+
+--------------------------------------------+
+--------------------------------------------+
(3, 4)
(array(['HOUR_OF_DAY', 'OBJECT_OF_CONVERSION_TRACKING', 'CONVERSION_NAME',
       'CLICK_TYPE'], dtype='<U29'), array(['IMPRESSION_SHARE', 'EXACT_MATCH_IMPRESSION_SHARE',
       'BUDGET_LOST_IMPRESSION_SHARE', 'QUALITY_LOST_IMPRESSION_SHARE',
       'INVALID_CLICKS', 'INVALID_CLICK_RATE', 'ALL_CONV_RATE',
       'COST_PER_ALL_CONV', 'COST', 'IMPS', 'CLICKS', 'CLICK_RATE',
       'AVG_CPC', 'AVG_DELIVER_RANK', 'CONV_RATE', 'COST_PER_CONV',
       'ALL_CONV', 'ALL_CONV_VALUE', 'VALUE_PER_ALL_CONV',
       'CROSS_DEVICE_CONVERSIONS'], dtype='<U29'))
24
(1, 2, 4)
(array(['HOUR_OF_DAY'], dtype='<U11'), array(['HOUR_OF_DAY', 'OBJECT_OF_CONVERSION_TRACKING', 'CONVERSION_NAME'],
      dtype='<U29'), array(['CL

In [250]:
for i in itertools.product([1,2,3], [1,2,3], [1,2,3]):
    print(i)

(1, 1, 1)
(1, 1, 2)
(1, 1, 3)
(1, 2, 1)
(1, 2, 2)
(1, 2, 3)
(1, 3, 1)
(1, 3, 2)
(1, 3, 3)
(2, 1, 1)
(2, 1, 2)
(2, 1, 3)
(2, 2, 1)
(2, 2, 2)
(2, 2, 3)
(2, 3, 1)
(2, 3, 2)
(2, 3, 3)
(3, 1, 1)
(3, 1, 2)
(3, 1, 3)
(3, 2, 1)
(3, 2, 2)
(3, 2, 3)
(3, 3, 1)
(3, 3, 2)
(3, 3, 3)
